In [2]:
import torch

In [3]:
from envs.othello.vectenv import OthelloVectEnv
from envs.othello.torchscripts import get_legal_actions, push_actions

env = OthelloVectEnv(100, torch.device('cpu'), 8)

/Users/marshingjay/Repos/lazyzero/envs/othello/torchscripts.py:84: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x+1 < states_size:
/Users/marshingjay/Repos/lazyzero/envs/othello/torchscripts.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  flips[f_index, y, x, y, x+1:min(x+i, states_size)] = 1
/Users/marshingjay/Repos/lazyzero/envs/othello/torchscripts.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in

In [11]:
env.reset()

legal_actions = get_legal_actions(env.states, env.ray_tensor)
actions = env.fast_weighted_sample(legal_actions.view(-1, 64).float())
actions

tensor([43, 20, 43, 43, 34, 43, 20, 43, 43, 43, 29, 29, 29, 20, 20, 20, 43, 43,
        43, 20, 29, 34, 34, 43, 29, 20, 43, 20, 34, 43, 34, 29, 29, 43, 29, 20,
        29, 20, 29, 29, 29, 29, 43, 34, 34, 34, 34, 29, 43, 29, 34, 20, 20, 34,
        20, 29, 29, 29, 43, 43, 34, 43, 34, 43, 20, 29, 20, 34, 29, 29, 20, 20,
        29, 29, 29, 29, 34, 34, 20, 20, 29, 34, 43, 34, 43, 43, 20, 29, 34, 34,
        29, 43, 20, 29, 43, 43, 43, 34, 43, 29])

In [13]:
push_actions(env.states, env.ray_tensor, actions)
pass